In [ ]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [60]:
import pandas as pd # for aristhmetic computing
import numpy as np 
import math
import datetime # for timing
import os,sys,gc # for debuging
import re, string
import nltk
from nltk.tokenize import RegexpTokenizer
import time
import re

# replacements
replacement_patterns = [
     (r'won\'t', 'will not'),
     (r'can\'t', 'cannot'),
     (r'i\'m', 'i am'),
     (r'ain\'t', 'is not'),
     (r'(\w+)\'ll', '\g<1> will'),
     (r'(\w+)n\'t', '\g<1> not'),
     (r'(\w+)\'ve', '\g<1> have'),
     (r'(\w+)\'s', '\g<1> is'),
     (r'(\w+)\'re', '\g<1> are'),
     (r'(\w+)\'d', '\g<1> would')
]
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s
# stop words
stopwords_set = set(stopwords.words('english'))
# lemmatization
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(sentence):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
# process rows
def ApplyBOW(ColValues):
    ''''''
    n = len(ColValues)
    result = ['' for i in range(n)]
    i = 0
    print('total length %s.' % n)
    start = time.time()
    for i in range(n):
        end = time.time()
        if(i % 5000 == 0):
            print('%s done, time elapsed %.2fs.' % (i, (end - start)))
        row = str(ColValues[i])
        # replacement
        replacer = RegexpReplacer()
        row = replacer.replace(row)
        #Remove Special Characters
        row = special_character_removal.sub('', row)
        #Replace Numbers
        row = replace_numbers.sub('n', row)
        # remove punctations
        tokenizer = RegexpTokenizer(r"[\w']+")
        if(row == np.nan):
            word_vec = []
        try:
            word_vec = tokenizer.tokenize(row)
        except Exception as e:
            print(type(row))
            sys.exit(1)
        # to lower
        word_vec = [word.lower() for word in word_vec]
        # remove stopwords
        word_vec = [word for word in word_vec if(word not in stopwords_set)]
        # lemmatization
        word_vec = lemmatize_all(word_vec)
        # length > 1
        word_vec = [w for w in word_vec if(len(w) > 1)]
        # join
        text = ' '.join(word_vec)
        result[i] = text
        i += 1
    return result
# data source   
DataBase = '../data'
DataSet = {
    'train': pd.read_csv('%s/raw/train.csv' % DataBase),
    'test': pd.read_csv('%s/raw/test.csv' % DataBase),
    #'sub': pd.read_csv('%s/raw/sample_submission.csv' % DataBase)
}
# save
OutputDir = '%s/clean' % DataBase
if(os.path.exists(OutputDir) == False):
    os.makedirs(OutputDir)
COMMENT = 'comment_text'
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for col in label_cols:
    DataSet['test'][col] = 0
# entry
for mod in ['train', 'test']:
    s = time.time()
    # fill null comment text
    DataSet[mod][COMMENT].fillna("", inplace= True)
    # process rows
    DataSet[mod][COMMENT] = ApplyBOW(DataSet[mod][COMMENT].values)
    # log1p length
    #DataSet[mod]['total_log1p_len'] = np.log1p(DataSet[mod][COMMENT].map(lambda x: len(x.split(' '))))
    # none label
    DataSet[mod]['none'] = 1 - DataSet[mod][label_cols].max(axis=1)
    # save
    DataSet[mod][:200].to_csv('%s/%s.csv' % (OutputDir, mod), index= False)
    DataUtil2.save(DataSet[mod], '%s/%s.%s' % (OutputDir, mod, 'hdf'), 'hdf')
    e = time.time()
    print('time %.2fs' % ((e - s)))

total length 95851.
0 done, time elapsed 0.00s.
5000 done, time elapsed 0.22s.
10000 done, time elapsed 0.45s.
15000 done, time elapsed 0.66s.
20000 done, time elapsed 0.86s.
25000 done, time elapsed 1.07s.
30000 done, time elapsed 1.28s.
35000 done, time elapsed 1.52s.
40000 done, time elapsed 1.74s.
45000 done, time elapsed 1.96s.
50000 done, time elapsed 2.17s.
55000 done, time elapsed 2.40s.
60000 done, time elapsed 2.61s.
65000 done, time elapsed 2.85s.
70000 done, time elapsed 3.06s.
75000 done, time elapsed 3.28s.
80000 done, time elapsed 3.50s.
85000 done, time elapsed 3.72s.
90000 done, time elapsed 3.95s.
95000 done, time elapsed 4.16s.
time 4.84s
total length 226998.
0 done, time elapsed 0.00s.
5000 done, time elapsed 0.25s.
10000 done, time elapsed 0.52s.
15000 done, time elapsed 0.78s.
20000 done, time elapsed 1.01s.
25000 done, time elapsed 1.25s.
30000 done, time elapsed 1.50s.
35000 done, time elapsed 1.76s.
40000 done, time elapsed 1.99s.
45000 done, time elapsed 2.23s